In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("../src")

In [3]:
from CompoTree import ComponentTree, Radicals, TSVariants, CharLexicon, IDC
from gensim.models.keyedvectors import KeyedVectors
import numpy as np
import numpy.linalg as la

In [4]:
kv = KeyedVectors.load("../data/fasttext-zh-sw/fasttext-zh-sw.kv")
kv.init_sims()

In [5]:
ctree = ComponentTree.load()
radicals = Radicals.load()
tsvars = TSVariants.load()
lexicon = CharLexicon.load()

In [6]:
ctree.query("慮", max_depth=3)

[<⿸:[<⿸:[<⿱:⺊②>][<⿻:['㇀']['乚']>]>][<⿱:['田']['心']>]>]

In [7]:
# heartRed_hits = ctree.find("忄", max_depth=3, bmp_only=True)
heartAll_hits = ctree.find("心", max_depth=3, bmp_only=True)


In [8]:
IDC.vert2

<IdcEnum.vert2: '⿱'>

In [9]:
from CompoTree import CTFounds
CTFounds(heartAll_hits).filter(idc=IDC.vert2.value, pos=1)

[('忞', [⿱1忞]),
 ('悡', [⿱1悡]),
 ('慾', [⿱1慾]),
 ('怨', [⿱1怨]),
 ('葾', [⿱1葾, ⿱1怨]),
 ('㤪', [⿱1㤪, ⿱1怨]),
 ('戆', [⿱1戆]),
 ('愳', [⿱1愳]),
 ('懯', [⿱1懯]),
 ('憊', [⿱1憊]),
 ('怂', [⿱1怂]),
 ('㤰', [⿱1㤰]),
 ('总', [⿱1总]),
 ('聪', [⿰1聪, ⿱1总]),
 ('搃', [⿰1搃, ⿱1总]),
 ('縂', [⿰1縂, ⿱1总]),
 ('恙', [⿱1恙]),
 ('㺊', [⿰1㺊, ⿱1恙]),
 ('䭐', [⿰1䭐(GT), ⿱1恙]),
 ('憝', [⿱1憝]),
 ('㥯', [⿱1㥯]),
 ('檼', [⿰1檼, ⿱1㥯]),
 ('隱', [⿰1隱, ⿱1㥯]),
 ('癮', [⿸1癮, ⿰1隱, ⿱1㥯]),
 ('䨸', [⿱1䨸, ⿰1隱, ⿱1㥯]),
 ('㦩', [⿰1㦩, ⿰1隱, ⿱1㥯]),
 ('㶏', [⿰1㶏, ⿰1隱, ⿱1㥯]),
 ('讔', [⿰1讔, ⿰1隱, ⿱1㥯]),
 ('櫽', [⿱0櫽(GJ), ⿰1隱, ⿱1㥯]),
 ('蘟', [⿱1蘟, ⿰1隱, ⿱1㥯]),
 ('䌥', [⿰1䌥(GT), ⿱1㥯]),
 ('䭡', [⿰1䭡, ⿱1㥯]),
 ('懚', [⿰1懚, ⿱1㥯]),
 ('濦', [⿰1濦, ⿱1㥯]),
 ('㡥', [⿰1㡥, ⿱1㥯]),
 ('嶾', [⿰1嶾, ⿱1㥯]),
 ('穩', [⿰1穩, ⿱1㥯]),
 ('㒚', [⿰1㒚, ⿱1㥯]),
 ('悉', [⿱1悉]),
 ('㴽', [⿰1㴽, ⿱1悉]),
 ('窸', [⿱1窸, ⿱1悉]),
 ('蟋', [⿰1蟋, ⿱1悉]),
 ('㣰', [⿰1㣰, ⿱1悉]),
 ('鏭', [⿰1鏭, ⿱1悉]),
 ('僁', [⿰1僁, ⿱1悉]),
 ('䊝', [⿰1䊝, ⿱1悉]),
 ('㗭', [⿰1㗭, ⿱1悉]),
 ('慗', [⿱1慗]),
 ('㤎', [⿱1㤎]),
 ('惪', [⿱1惪]),
 ('㦂', [⿱1㦂]),
 ('怣', [⿱1怣]),
 ('戀', [⿱1戀])

In [10]:
heartAll_hits

[('忞', [⿱1忞]),
 ('悡', [⿱1悡]),
 ('慾', [⿱1慾]),
 ('鈊', [⿰1鈊]),
 ('怨', [⿱1怨]),
 ('葾', [⿱1葾, ⿱1怨]),
 ('㤪', [⿱1㤪, ⿱1怨]),
 ('戆', [⿱1戆]),
 ('愳', [⿱1愳]),
 ('懯', [⿱1懯]),
 ('憊', [⿱1憊]),
 ('怂', [⿱1怂]),
 ('㤰', [⿱1㤰]),
 ('总', [⿱1总]),
 ('聪', [⿰1聪, ⿱1总]),
 ('搃', [⿰1搃, ⿱1总]),
 ('縂', [⿰1縂, ⿱1总]),
 ('恙', [⿱1恙]),
 ('㺊', [⿰1㺊, ⿱1恙]),
 ('䭐', [⿰1䭐(GT), ⿱1恙]),
 ('憝', [⿱1憝]),
 ('㥯', [⿱1㥯]),
 ('檼', [⿰1檼, ⿱1㥯]),
 ('隱', [⿰1隱, ⿱1㥯]),
 ('癮', [⿸1癮, ⿰1隱, ⿱1㥯]),
 ('䨸', [⿱1䨸, ⿰1隱, ⿱1㥯]),
 ('㦩', [⿰1㦩, ⿰1隱, ⿱1㥯]),
 ('㶏', [⿰1㶏, ⿰1隱, ⿱1㥯]),
 ('讔', [⿰1讔, ⿰1隱, ⿱1㥯]),
 ('櫽', [⿱0櫽(GJ), ⿰1隱, ⿱1㥯]),
 ('蘟', [⿱1蘟, ⿰1隱, ⿱1㥯]),
 ('䌥', [⿰1䌥(GT), ⿱1㥯]),
 ('䭡', [⿰1䭡, ⿱1㥯]),
 ('懚', [⿰1懚, ⿱1㥯]),
 ('濦', [⿰1濦, ⿱1㥯]),
 ('㡥', [⿰1㡥, ⿱1㥯]),
 ('嶾', [⿰1嶾, ⿱1㥯]),
 ('穩', [⿰1穩, ⿱1㥯]),
 ('㒚', [⿰1㒚, ⿱1㥯]),
 ('悉', [⿱1悉]),
 ('㴽', [⿰1㴽, ⿱1悉]),
 ('窸', [⿱1窸, ⿱1悉]),
 ('蟋', [⿰1蟋, ⿱1悉]),
 ('㣰', [⿰1㣰, ⿱1悉]),
 ('鏭', [⿰1鏭, ⿱1悉]),
 ('僁', [⿰1僁, ⿱1悉]),
 ('䊝', [⿰1䊝, ⿱1悉]),
 ('㗭', [⿰1㗭, ⿱1悉]),
 ('慗', [⿱1慗]),
 ('㤎', [⿱1㤎]),
 ('惪', [⿱1惪]),
 ('㦂', [⿱1㦂]),
 ('怣', [⿱1怣])

In [51]:
heartRed_hits
heartAll_hits

[('愈', [⿱1愈(GTJV)]),
 ('恁', [⿱1恁]),
 ('惄', [⿱1惄]),
 ('㥁', [⿱1㥁]),
 ('愻', [⿱1愻]),
 ('憊', [⿱1憊]),
 ('怠', [⿱1怠]),
 ('戇', [⿱1戇]),
 ('懟', [⿱1懟]),
 ('孞', [⿱1孞]),
 ('忐', [⿱1忐]),
 ('惉', [⿱1惉]),
 ('恧', [⿱1恧]),
 ('慤', [⿱1慤]),
 ('㥕', [⿱1㥕]),
 ('态', [⿱1态]),
 ('愗', [⿱1愗]),
 ('憵', [⿱1憵]),
 ('懯', [⿱1懯]),
 ('吣', [⿰1吣]),
 ('㥋', [⿱1㥋]),
 ('憗', [⿱1憗]),
 ('態', [⿱1態]),
 ('㤅', [⿱1㤅]),
 ('忈', [⿱1忈]),
 ('窓', [⿳2窓]),
 ('怨', [⿱1怨]),
 ('懕', [⿸1懕(GT)]),
 ('恝', [⿱1恝]),
 ('㥣', [⿱1㥣]),
 ('㣻', [⿱1㣻]),
 ('㤫', [⿱1㤫]),
 ('慦', [⿱1慦]),
 ('㥻', [⿱1㥻]),
 ('慿', [⿱1慿]),
 ('恴', [⿱1恴]),
 ('伈', [⿰1伈]),
 ('慸', [⿱1慸]),
 ('愬', [⿱1愬]),
 ('恚', [⿱1恚]),
 ('感', [⿱1感(GJK)]),
 ('您', [⿱1您]),
 ('息', [⿱1息]),
 ('沁', [⿰1沁]),
 ('鈊', [⿰1鈊]),
 ('必', [⿻0必]),
 ('愍', [⿱1愍]),
 ('懖', [⿱1懖]),
 ('㥈', [⿱1㥈]),
 ('㦝', [⿱1㦝]),
 ('忘', [⿱1忘]),
 ('惒', [⿱1惒]),
 ('恙', [⿱1恙]),
 ('悶', [⿵1悶]),
 ('憅', [⿱1憅]),
 ('懘', [⿱1懘]),
 ('恕', [⿱1恕]),
 ('悳', [⿱1悳]),
 ('想', [⿱1想]),
 ('慇', [⿱1慇]),
 ('慧', [⿱1慧]),
 ('愳', [⿱1愳]),
 ('㥹', [⿱1㥹]),
 ('憋', [⿱1憋]),
 ('总', [⿱1总]),
 ('惥', [⿱1

In [60]:
heartRed_char = [x[0] for x in heartRed_hits if x[0] in kv.vocab]
heartAll_char = [x[0] for x in heartAll_hits if x[0] in kv.vocab]

In [63]:
heartRed_char[1]

'愫'

In [85]:
heartRed_trad = []
heartRed_simp = []
heartRed_share = []
for w in heartRed_char:
    if tsvars.is_traditional(w) and tsvars.is_simplified(w) == False:
        heartRed_trad.append(w)
    elif tsvars.is_simplified(w) and tsvars.is_traditional(w) == False:
        heartRed_simp.append(w)
    else:
        heartRed_share.append(w)

In [89]:
heartAll_trad = []
heartAll_simp = []
heartAll_share = []
for w in heartAll_char:
    if tsvars.is_traditional(w) and tsvars.is_simplified(w) == False:
        heartAll_trad.append(w)
    elif tsvars.is_simplified(w) and tsvars.is_traditional(w) == False:
        heartAll_simp.append(w)
    else:
        heartAll_share.append(w)

In [92]:
print(heartAll_simp)

['态', '总', '怂', '虑', '戆', '恋', '慭', '悫', '惩', '闷', '懑', '怼', '悬', '恳', '恶']


In [93]:
print(heartAll_trad)

['憊', '戇', '懟', '態', '悶', '惡', '憑', '懣', '應', '懲', '愨', '懸', '懇', '戀', '憲', '恥']


In [84]:
tsvars.is_simplified('志')

True

In [77]:
heart_sv = kv.vectors_norm[kv.vocab['心'].index,:]

In [109]:
heartRed_trad_cor = [kv.similarity('心', w) for w in heartRed_trad]
heartAll_trad_cor = [kv.similarity('心', w) for w in heartAll_trad]
heartRed_simp_cor = [kv.similarity('心', w) for w in heartRed_simp]
heartAll_simp_cor = [kv.similarity('心', w) for w in heartAll_simp]
heartRed_share_cor = [kv.similarity('心', w) for w in heartRed_share]
heartAll_share_cor = [kv.similarity('心', w) for w in heartAll_share]

In [110]:
np.mean(heartRed_trad_cor), np.mean(heartAll_trad_cor)

(0.946144316613844, 0.9481927658387708)

In [111]:
np.mean(heartRed_simp_cor), np.mean(heartAll_simp_cor)

(0.9502067022170306, 0.9462629050521761)

In [112]:
np.mean(heartRed_share_cor), np.mean(heartAll_share_cor)

(0.945840444409698, 0.9523935354593236)

In [32]:
kv.vocab

{'的': <gensim.models.keyedvectors.Vocab at 0x7fe085fc0828>,
 '年': <gensim.models.keyedvectors.Vocab at 0x7fe06a48dc18>,
 '月': <gensim.models.keyedvectors.Vocab at 0x7fe06a48dc50>,
 '日': <gensim.models.keyedvectors.Vocab at 0x7fe06a48dc88>,
 '在': <gensim.models.keyedvectors.Vocab at 0x7fe06a48dcc0>,
 '是': <gensim.models.keyedvectors.Vocab at 0x7fe06a48dcf8>,
 '和': <gensim.models.keyedvectors.Vocab at 0x7fe06a48dd30>,
 '了': <gensim.models.keyedvectors.Vocab at 0x7fe06a48dd68>,
 '村': <gensim.models.keyedvectors.Vocab at 0x7fe06a48dda0>,
 '人': <gensim.models.keyedvectors.Vocab at 0x7fe06a48ddd8>,
 '中': <gensim.models.keyedvectors.Vocab at 0x7fe06a48de10>,
 '为': <gensim.models.keyedvectors.Vocab at 0x7fe06a48de48>,
 '為': <gensim.models.keyedvectors.Vocab at 0x7fe06a48de80>,
 '被': <gensim.models.keyedvectors.Vocab at 0x7fe06a48deb8>,
 '斯': <gensim.models.keyedvectors.Vocab at 0x7fe06a48def0>,
 '有': <gensim.models.keyedvectors.Vocab at 0x7fe06a48df28>,
 '及': <gensim.models.keyedvectors.Vocab 